In [5]:
import json
import re

In [45]:
import json 

with open("dataset.json", "r") as file:
    obj = json.load(file)
    
print("Len: ", len(obj))
print(obj[:3])

Len:  30
["def encrypt(string, key):\n    str = [(ord(chr) - ord('A') + key) % 26 + ord('A') if chr.isupper() == 1\n        else (ord(chr) - ord('a') + key) % 26 + ord('a') if chr.islower() else ord(chr) for chr in string]\n    print(str)\n    return ''.join([chr(ch) for ch in str])", 'def check_prime(x):\n        if (x < 2):\n            return False\n        if (x == 2):\n             return True\n        for k in range(3, int(math.sqrt(x)), 2):\n            if x % k == 0:\n                return False \n        return True', "for word in content.split(' '):\n    word = ''.join([ch for ch in word if ch.isalpha()])\n    freq[word] = freq.get(word,  0) + 1"]


In [46]:
import re
def split_code(code):
    return re.findall(r'\w+|[^\w]', code)

obj = [a.splitlines() for a in obj]
obj

[['def encrypt(string, key):',
  "    str = [(ord(chr) - ord('A') + key) % 26 + ord('A') if chr.isupper() == 1",
  "        else (ord(chr) - ord('a') + key) % 26 + ord('a') if chr.islower() else ord(chr) for chr in string]",
  '    print(str)',
  "    return ''.join([chr(ch) for ch in str])"],
 ['def check_prime(x):',
  '        if (x < 2):',
  '            return False',
  '        if (x == 2):',
  '             return True',
  '        for k in range(3, int(math.sqrt(x)), 2):',
  '            if x % k == 0:',
  '                return False ',
  '        return True'],
 ["for word in content.split(' '):",
  "    word = ''.join([ch for ch in word if ch.isalpha()])",
  '    freq[word] = freq.get(word,  0) + 1'],
 ['@app.route("/")',
  '    def index(name=None):',
  '    return render_template("initial_design.html", name=name)'],
 ['@app.route("/login", methods=["GET", "POST"])',
  'def login():',
  '    error_msg = ""',
  '    if request.method == "POST":',
  '        username = reques

In [60]:
import random
ratio = 0.15 

cs = []

for code in obj: 
    l = len(code)
    a = random.randint(1, max(l-2, 1))
    
    cs.append(("\n".join(code[:a]), "\n".join(code[a:a+1]), "\n".join(code[a+1:])))

cs

[('def encrypt(string, key):',
  "    str = [(ord(chr) - ord('A') + key) % 26 + ord('A') if chr.isupper() == 1",
  "        else (ord(chr) - ord('a') + key) % 26 + ord('a') if chr.islower() else ord(chr) for chr in string]\n    print(str)\n    return ''.join([chr(ch) for ch in str])"),
 ('def check_prime(x):\n        if (x < 2):\n            return False\n        if (x == 2):\n             return True\n        for k in range(3, int(math.sqrt(x)), 2):',
  '            if x % k == 0:',
  '                return False \n        return True'),
 ("for word in content.split(' '):",
  "    word = ''.join([ch for ch in word if ch.isalpha()])",
  '    freq[word] = freq.get(word,  0) + 1'),
 ('@app.route("/")',
  '    def index(name=None):',
  '    return render_template("initial_design.html", name=name)'),
 ('@app.route("/login", methods=["GET", "POST"])\ndef login():',
  '    error_msg = ""',
  '    if request.method == "POST":\n        username = request.form.get("username", "")\n        pass

In [61]:
splitted_code = json.dumps(cs)

with open("splitted2.json", "w") as file:
    file.write(splitted_code)
    

In [ ]:
!pip install transformers
!pip install torch

In [6]:
import json 

with open("splitted2.json", "r") as file:
    cs = json.load(file)
    
cs

[['def encrypt(string, key):',
  "    str = [(ord(chr) - ord('A') + key) % 26 + ord('A') if chr.isupper() == 1",
  "        else (ord(chr) - ord('a') + key) % 26 + ord('a') if chr.islower() else ord(chr) for chr in string]\n    print(str)\n    return ''.join([chr(ch) for ch in str])"],
 ['def check_prime(x):\n        if (x < 2):\n            return False\n        if (x == 2):\n             return True\n        for k in range(3, int(math.sqrt(x)), 2):',
  '            if x % k == 0:',
  '                return False \n        return True'],
 ["for word in content.split(' '):",
  "    word = ''.join([ch for ch in word if ch.isalpha()])",
  '    freq[word] = freq.get(word,  0) + 1'],
 ['@app.route("/")',
  '    def index(name=None):',
  '    return render_template("initial_design.html", name=name)'],
 ['@app.route("/login", methods=["GET", "POST"])\ndef login():',
  '    error_msg = ""',
  '    if request.method == "POST":\n        username = request.form.get("username", "")\n        pass

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import csv

checkpoint = "bigcode/tiny_starcoder_py"
suffix = "<fim_suffix>"
prefix = "<fim_prefix>"
middle = "<fim_middle>"
eot = "<|endoftext|>"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint)
out = ["Prefix", "Suffix", "Expected Middle", "Actual Middle"]
file = open("output.csv", "w")
wrt = csv.writer(file)
wrt.writerow(out)

notcsv = []

for snippet in cs:
    input = prefix + snippet[0] + suffix + snippet[2] + middle
    inputs = tokenizer.encode(input, return_tensors="pt")
    outputs = model.generate(inputs, max_new_tokens=20)
    out = tokenizer.decode(outputs[0])
    notcsv.append([snippet[0], snippet[2], snippet[1], out[out.find(middle) + len(middle):]])

objc = json.dumps(notcsv)

with open("output.json", "w") as file:
    file.write(objc)

In [ ]:
!pip install Levenshtein

In [10]:
from collections import Counter
import Levenshtein

def character_fscore(predicted, ground_truth):
    predicted_counter = Counter(predicted)
    ground_truth_counter = Counter(ground_truth)
    
    true_positives = sum((predicted_counter & ground_truth_counter).values())
    false_positives = sum((predicted_counter - ground_truth_counter).values())
    false_negatives = sum((ground_truth_counter - predicted_counter).values())
    
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    
    if precision + recall == 0:
        f1_score = 0
    else:
        f1_score = 2 * (precision * recall) / (precision + recall)
    
    return precision, recall, f1_score

def is_contained(predicted, ground_truth):
    return True if predicted in ground_truth else False 

def levenshtein(predicted, ground_truth):
    return Levenshtein.distance(predicted, ground_truth)


def jaro_similarity(s1, s2):
    # Lengths of both strings
    len_s1 = len(s1)
    len_s2 = len(s2)

    if len_s1 == 0 and len_s2 == 0:
        return 1.0  # Both strings are empty, so similarity is 1.0

    match_distance = max(len_s1, len_s2) // 2 - 1

    # Lists to store matches
    s1_matches = [False] * len_s1
    s2_matches = [False] * len_s2

    # Count matches
    matches = 0
    for i in range(len_s1):
        start = max(0, i - match_distance)
        end = min(i + match_distance + 1, len_s2)
        for j in range(start, end):
            if s2_matches[j]:
                continue
            if s1[i] == s2[j]:
                s1_matches[i] = True
                s2_matches[j] = True
                matches += 1
                break

    if matches == 0:
        return 0.0

    # Count transpositions
    transpositions = 0
    s2_index = 0
    for i in range(len_s1):
        if s1_matches[i]:
            while not s2_matches[s2_index]:
                s2_index += 1
            if s1[i] != s2[s2_index]:
                transpositions += 1
            s2_index += 1

    transpositions /= 2

    # Calculate Jaro similarity
    jaro = (matches / len_s1 + matches / len_s2 + (matches - transpositions) / matches) / 3
    return jaro

def jaro_winkler_similarity(s1, s2, p=0.1, max_prefix_length=4):
    jaro_sim = jaro_similarity(s1, s2)

    # Find the length of common prefix
    prefix_length = 0
    for i in range(min(len(s1), len(s2), max_prefix_length)):
        if s1[i] == s2[i]:
            prefix_length += 1
        else:
            break

    # Jaro-Winkler adjustment
    jaro_winkler_sim = jaro_sim + (prefix_length * p * (1 - jaro_sim))

    return jaro_winkler_sim

In [11]:
with open("output.json") as file:
    notcsv = json.load(file)

In [12]:
notcsv[0]

['def encrypt(string, key):',
 "        else (ord(chr) - ord('a') + key) % 26 + ord('a') if chr.islower() else ord(chr) for chr in string]\n    print(str)\n    return ''.join([chr(ch) for ch in str])",
 "    str = [(ord(chr) - ord('A') + key) % 26 + ord('A') if chr.isupper() == 1",
 '\n    if len(string) < 26:\n        return string\n    if len(string)']

In [17]:
with open("analysis.txt", "w") as file:
    
    for cs in notcsv:
        file.write("\n================ Code Snippet =====================\n")
        file.write("\n=================== Prefix ========================\n")
        file.write("\n'''\n")
        file.write(cs[0])
        file.write("\n'''\n")
        file.write("\n================== Expected =======================\n")
        file.write("\n'''\n")
        file.write(cs[2])
        file.write("\n'''\n")
        file.write("\n================ Model Result =====================\n")
        cs[3] = str(re.sub(r'[\u4e00-\u9fff]+', '', cs[3]))
        file.write("\n'''\n")
        file.write(cs[3])
        file.write("\n'''\n")
        file.write("\n================== Suffix =========================\n")
        file.write("\n'''\n")
        file.write(cs[1])
        file.write("\n'''\n")
        file.write("\n================== Metrics =========================\n")
        file.write("\n")
        file.write("Exact Match: " + str(cs[2] == cs[3]))
        file.write("\n")
        file.write("Is contained: ")
        file.write(str(is_contained(cs[2], cs[3])))
        file.write("\n")
        file.write("\n")
        p, r, f = character_fscore(cs[2], cs[3])
        file.write("chrf precision:")
        file.write(str(p))
        file.write("\n")
        file.write("chrf recall:")
        file.write(str(r))
        file.write("\n")
        file.write("chrf f1:")
        file.write(str(f))
        file.write("\n")
        file.write("\n")
        file.write("Levenshtein Distance: ")
        file.write(str(levenshtein(cs[2], cs[3])))
        file.write("\nJaro-Winkler Similarity: ")
        file.write(str(jaro_winkler_similarity(cs[2], cs[3])))
        file.write("\n\nManual Review:\n ")
        file.write("\n")
        file.write("\n\n\n\n\n\n\n\n")